# Заполнение отчета по доходам за рубежом на nalog.ru

## Автор

Alexander Gerasiov <a@gerasiov.net>

http://github.com/gerasiov/3ndfl

## Лицензия

GPL версии 2 или более поздняя

## Подробнее

Читайте в README.md

In [ ]:
# Install selenium into venv or user's dir

#!pip install selenium
#!pip install --user selenium

# Or install is manually to your system with
# sudo apt install python3-selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from decimal import Decimal
import getpass
import csv

In [ ]:
driver = None

def open_browser(executable):
    global driver
    options = webdriver.chrome.options.Options()

    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_argument('--incognito')
 
    prefs = {
        'safebrowsing.enabled': True,
        'credentials_enable_service': False,
        'profile.password_manager_enabled' : False,
        'credentials_enable_service': False,
        'profile.password_manager_enabled': False
    }
    options.add_experimental_option('prefs', prefs)
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-infobars')
    options.add_argument('--safebrowsing-disable-extension-blacklist')
    options.add_argument('--safebrowsing-disable-download-protection')


    service = webdriver.chrome.service.Service(executable_path=executable)
    driver = webdriver.Chrome(service=service, options=options)

In [ ]:
def get_button_with_text(text):
    return f'//button[contains(.,"{text}")]'


def wait_for_elemet(path, retry=10):
    for ret in range(1, retry + 1):
        try:
            driver.find_element(by=By.XPATH, value=path)
            break
        except NoSuchElementException:
            if ret == retry:
                raise
            driver.implicitly_wait(1)

def move_to_element(e):
    ActionChains(driver).move_to_element(e).perform()
            

def click_button(path):
    wait_for_elemet(path)
    button = driver.find_element(by=By.XPATH, value=path)
    move_to_element(button)
    button.click()

In [ ]:
def login():
    login_url = 'https://lkfl2.nalog.ru/lkfl/login'
    logged_in = 'https://lkfl2.nalog.ru/lkfl/individual/main'
    driver.get(login_url)
    driver.implicitly_wait(0.5)
    if driver.current_url != logged_in:
        click_button(get_button_with_text('ЕСИА'))
        driver.implicitly_wait(1)
        if driver.current_url != logged_in:
            print('Please login in the browser window')
            while driver.current_url != logged_in:
                driver.implicitly_wait(3)
    
    print('Logged in, proceed to next step.')

In [ ]:
def click_next():
    click_button(get_button_with_text('Далее'))

In [ ]:
def open_3nfdl_foreing_income_form():
    driver.get('https://lkfl2.nalog.ru/lkfl/individual/appeals/3NDFL/3NDFL')
    driver.implicitly_wait(0.5)

    click_next()
    click_button(get_button_with_text('За пределами РФ'))
    driver.implicitly_wait(0.5)


    # Hide opaque header
    while True:
        try:
            element = driver.find_element(by=By.XPATH, value=f'//div[starts-with(@class,"sticky")]')
        except:
            break
        driver.execute_script("""
        var element = arguments[0];
        element.parentNode.removeChild(element);
        """, element)

In [ ]:
def fill_text(field, value):
    i=driver.find_element(by=By.XPATH, value=f'//input[@name="{field}"]')
    move_to_element(i)
    i.send_keys(Keys.CONTROL,"a")  # Have to delete manually because of "smart" JS
    i.send_keys(Keys.DELETE)
    i.clear()
    i.send_keys(value)


def fill_date(field, value):
    i=driver.find_element(by=By.XPATH, value=f'//input[starts-with(@id,"{field}")]')
    move_to_element(i)
    i.send_keys(Keys.CONTROL,"a")  # Have to delete manually because of "smart" JS
    i.send_keys(Keys.DELETE)
    i.clear()
    i.send_keys(value, Keys.ENTER)
    i.click()


def fill_dropdown(field, value):
    i=driver.find_element(by=By.XPATH, value=f'//input[@name="{field}"]')
    move_to_element(i)
    i.click()
    i.send_keys(value)
    driver.implicitly_wait(0.5)
    i=driver.find_element(by=By.XPATH, value='//li[starts-with(@class,"fns-select__option")]')
    i.click()


def click_currency_online_checkbox():
    i=driver.find_elements(
        by=By.XPATH, 
        value=f'//input[starts-with(@id, "module-checkbox-")]')[-1]
    move_to_element(i)
    i.click()

In [ ]:
CURRENT_INCOME_TAB = '//div[@class="fns-tabs__tabPanels"]/div[not(@hidden)]'
FOREING_INCOME_PREFIX = 'payload.sheetB.sources.'

def add_income(source_name, source_country, dest_country, amount, income_date, income_type, currency, income_tax):
    click_button(CURRENT_INCOME_TAB+get_button_with_text('Добавить источник дохода'))

    fill_text('incomeSourceName', source_name)
    fill_dropdown('oksmIst', source_country)
    fill_dropdown('oksmZach', dest_country)

    click_button('//form'+get_button_with_text('Добавить'))
    
    income_number = len(set(
        [e.get_attribute('name').split('.')[3] for e in driver.find_elements(
            by=By.XPATH,
            value=f'//input[starts-with(@name, "{FOREING_INCOME_PREFIX}")]')])) - 1 
    
    
    fill_dropdown(f'{FOREING_INCOME_PREFIX}{income_number}.incomeTypeCode', income_type)
    fill_dropdown(f'{FOREING_INCOME_PREFIX}{income_number}.taxDeductionCode', 'Не предоставлять')
    fill_text(f'{FOREING_INCOME_PREFIX}{income_number}.incomeAmountCurrency', amount)
    fill_date(f'{FOREING_INCOME_PREFIX}{income_number}.incomeDate', income_date)    
    fill_dropdown(f'{FOREING_INCOME_PREFIX}{income_number}.currencyCode', currency)
    if income_tax:
        fill_date(f'{FOREING_INCOME_PREFIX}{income_number}.taxPaymentDate', income_date)
        fill_text(f'{FOREING_INCOME_PREFIX}{income_number}.paymentAmountCurrency',income_tax) 
    
    
    click_currency_online_checkbox()

In [ ]:
# Mappings from country/currency name abbrv. to OKSM code

def get_country_code(country):
    return {
        'Россия': 643,
        'РФ': 643,
        'США': 840,
        'Германия': 276,
        'ДЖЕРСИ': 832,
        'Нидерланды': 528,
        'Израиль': 376,
        'Казахстан': 398
    }[country]


def get_currency_code(currency):
    return {
        'RUB': 643,
        'USD': 840,
        'EUR': 978,
        'NIS': 376,
        'KZT': 398
    }[currency]


def get_income_code(t):
    return {
        'divident': 1010,
        'salary': 2000,
        'insurance': 1200,
    }[t]

In [ ]:
# Should return list of dicts e.g
# [
#     {
#         'date': '01.01.2021',
#         'name': 'Yandex B.V.',
#         'type': 'divident',
#         'src_country': 'Нидерланды',
#         'dst_country': 'РФ',
#         'currency': 'USD',
#         'amount': '20.20',
#         'tax': '0.20'
#     },
# ]
def read_report(filename):
    result = []
    with open(filename, 'r') as file:        
        fields = ['date', 'src_country', 'dst_country', 'type', 'currency', 'amount', 'tax', 'name']
        tsv_file = csv.DictReader(file, fieldnames=fields, delimiter="\t")
        for line in tsv_file:
            result.append(line)
    return result

In [ ]:
# Pass full name of driver's executable
# Note, that in Debian/Ubuntu you should install it with chromium-chromedriver package
open_browser(executable='/usr/bin/chromedriver')

На следующем шаге откроется окно браузера, где необходимо будет залогиниться на сайте налоговой.

In [ ]:
login()

In [ ]:
open_3nfdl_foreing_income_form()

In [ ]:
for record in read_report('income.tsv'):
    add_income(
        source_name=record['name'],
        source_country=get_country_code(record['src_country']),
        dest_country=get_country_code(record['dst_country']),
        amount=record['amount'],
        income_date=record['date'],
        income_type=get_income_code(record['type']),
        currency=get_currency_code(record['currency']),
        income_tax=record['tax']
    )

In [ ]:
# This one may fail if you have not filled some required fields on other tabs
click_next()